In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bem-vindo ao curso **Limpeza de dados** no Kaggle Learn!

A limpeza de dados é uma parte fundamental da ciência de dados, mas pode ser profundamente frustrante. Por que alguns dos seus campos de texto estão distorcidos? O que você deve fazer com relação a esses valores ausentes? Por que suas datas não estão formatadas corretamente? Como você pode limpar rapidamente a entrada de dados inconsistentes? Neste curso, você aprenderá por que se deparou com esses problemas e, mais importante, como resolvê-los!

Neste curso, você aprenderá como resolver alguns dos problemas mais comuns de limpeza de dados para que possa analisar seus dados com mais rapidez. Você realizará cinco exercícios práticos com dados reais e confusos e responderá a algumas das perguntas mais frequentes sobre limpeza de dados.

Neste caderno, veremos como lidar com valores ausentes.
O dataset está disponível [aqui](https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016)

# Dê uma primeira olhada nos dados

A primeira coisa que precisaremos fazer é carregar as bibliotecas e o conjunto de dados que usaremos.

Para demonstração, usaremos um conjunto de dados de eventos que ocorreram em jogos de futebol americano.

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
jogo_futebol = pd.read_csv("/content/drive/MyDrive/Kaggle_course/NFL Play by Play 2009-2017 (v4).csv")

# set seed for reproducibility
np.random.seed(0)

<ipython-input-3-d6dcdeed56a0>:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  jogo_futebol = pd.read_csv("/content/drive/MyDrive/Kaggle_course/NFL Play by Play 2009-2017 (v4).csv")


A primeira coisa a fazer quando você obtém um novo conjunto de dados é dar uma olhada em alguns deles. Isso permite que você veja se tudo foi lido corretamente e dá uma ideia do que está acontecendo com os dados. Neste caso, vamos ver se há algum valor faltante, que será representado por `NaN` ou `None`.

In [ ]:
# observe as primeiras cinco linhas do arquivo jogo_futebol.
# Já consigo ver alguns dados faltantes!
jogo_futebol.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Sim, parece que faltam alguns valores.

# Quantos pontos de dados faltantes temos?

Ok, agora sabemos que temos alguns valores ausentes. Vamos ver quantos temos em cada coluna.

In [ ]:
# obtém o número de pontos de dados ausentes por coluna
missing_values_count = jogo_futebol.isnull().sum()

# veja o número de pontos faltantes nas primeiras dez colunas
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Isso parece muito! Pode ser útil ver qual a porcentagem dos valores em nosso conjunto de dados que esta faltando para nos dar uma noção melhor da escala deste problema:

In [ ]:
# quantos valores faltantes totais temos?
total_cells = np.product(jogo_futebol.shape)
total_missing = missing_values_count.sum()

# porcentagem dos dados faltantes:
percent_missing = (total_missing/total_cells) * 100
print(f'{percent_missing:.2f}%')

27.67%


Uau, quase um quarto das células deste conjunto de dados estão vazias! Na próxima etapa, examinaremos mais de perto algumas das colunas com valores ausentes e tentaremos descobrir o que pode estar acontecendo com elas.

# Descubra por que os dados estão faltando

Este é o ponto em que entramos na parte da ciência de dados que gosto de chamar de “intuição de dados”, com o que quero dizer “realmente olhar para seus dados e tentar descobrir por que eles são do jeito que são e como isso vai acontecer”. afetar sua análise". Pode ser uma parte frustrante da ciência de dados, especialmente se você for novo na área e não tiver muita experiência. Para lidar com valores ausentes, você precisará usar sua intuição para descobrir por que o valor está ausente. Uma das perguntas mais importantes que você pode se fazer para ajudar a descobrir isso é esta:

> **Este valor está faltando porque não foi registrado ou porque não existe?**

Se um valor estiver faltando porque não existe (como a altura do filho mais velho de alguém que não tem filhos), então não faz sentido tentar adivinhar o que pode ser. Você provavelmente deseja manter esses valores como `NaN`. Por outro lado, se um valor estiver faltando porque não foi registrado, você poderá tentar adivinhar o que poderia ter sido com base nos outros valores dessa coluna e linha. Isso se chama **imputação** e aprenderemos como fazer isso a seguir! :)

Vamos trabalhar com um exemplo. Observando o número de valores ausentes no dataframe `nfl_data`, percebo que a coluna "TimesSec" contém muitos valores ausentes:

In [ ]:
# veja o número de pontos faltantes nas primeiras dez colunas
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Olhando a [documentação](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), posso ver que esta coluna contém informações sobre o número de segundos restantes do jogo quando a jogada foi feita. Isto significa que estes valores provavelmente estão faltando porque não foram registrados, e não porque não existem. Portanto, faria sentido tentarmos adivinhar o que deveriam ser, em vez de apenas deixá-los como NA.

Por outro lado, existem outros campos, como "PenalizedTeam" que também possuem muitos campos faltantes. Neste caso, porém, falta campo porque se não houve penalidade não faz sentido dizer *qual* time foi penalizado. Para esta coluna, faria mais sentido deixá-la vazia ou adicionar um terceiro valor como “nenhum” e usá-lo para substituir os NAs.

Se você estiver fazendo uma análise de dados muito cuidadosa, este é o ponto em que você examinará cada coluna individualmente para descobrir a melhor estratégia para preencher esses valores ausentes. No restante deste notebook, abordaremos algumas técnicas "rápidas e sujas" que podem ajudá-lo com valores ausentes, mas provavelmente também acabarão removendo algumas informações úteis ou adicionando algum ruído aos seus dados.

# Eliminar valores ausentes

Se você estiver com pressa ou não tiver um motivo para descobrir por que seus valores estão faltando, uma opção que você tem é simplesmente remover quaisquer linhas ou colunas que contenham valores faltantes. (Observação: geralmente não é recomendado essa abordagem para projetos importantes! Geralmente vale a pena dedicar um tempo para examinar seus dados e realmente observar todas as colunas com valores ausentes, uma por uma, para realmente conhecer seu conjunto de dados .)

Se você tem certeza de que deseja eliminar linhas com valores ausentes, o pandas tem uma função útil, `dropna()` para ajudá-lo a fazer isso. Vamos experimentar em nosso conjunto de dados da NFL!

In [ ]:
#remove todas as linhas que contém um valor faltante
jogo_futebol.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Nossa, parece que isso removeu todos os nossos dados! 😱 Isso ocorre porque cada linha em nosso conjunto de dados tinha pelo menos um valor ausente. Poderíamos ter mais sorte removendo todas as *colunas* que possuem pelo menos um valor ausente.

In [ ]:
#remove todas as colunas com pelo menos um valor faltando
columns_with_na_dropped = jogo_futebol.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [ ]:
# quantos dados perdemos?
print("Columns in original dataset: %d \n" % jogo_futebol.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 37


Perdemos muitos dados, mas neste ponto removemos com sucesso todos os `NaN` de nossos dados.

# Preenchendo valores faltantes automaticamente

Outra opção é tentar preencher os valores ausentes. Para a próxima parte, estou obtendo uma pequena subseção dos dados da NFL para que sejam bem impressos.

In [ ]:
#obtém um pequeno subconjunto do conjunto de dados da NFL
subset_nfl_data = jogo_futebol.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Podemos usar a função `fillna()` do Pandas para preencher os valores ausentes em um dataframe para nós. Uma opção que temos é especificar com o que queremos que os valores `NaN` sejam substituídos. Aqui, estou dizendo que gostaria de substituir todos os valores `NaN` por 0.

In [ ]:
#substitua todos os NaN's por 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


Eu também poderia ser um pouco mais experiente e substituir os valores ausentes por qualquer valor que venha diretamente depois dele na mesma coluna. (Isso faz muito sentido para conjuntos de dados onde as observações têm algum tipo de ordem lógica.)

In [ ]:
# substitua todos os NaN's pelo valor que vem diretamente depois dele na mesma coluna,
# então substitua todos os NaN's restantes por 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009
